# IMDS Computer Workshop 9 
### *By Jeffrey Giansiracusa - Michaelmas 2023*

---

This worksheet covers the content of lectures

    (some material from last week's videos)
    9.1 Rabbit model part 1
    9.2 Rabbit model part 2: long term behaviour
    
    (and some content from this week's videos)
    8.1 Eigenvectors
    8.2 Eigen examples

Key ideas to learn:

* Know the definition of eigenvectors and eigenvalues and how to get the computer to find them.
* Be able to formulate, simulate, and interpret discrete linear dynamical system models such as the rabbit model from  the lectures (and variations)
* How eigenvectors determine the long-time behaviour of discrete linear dynamical systems.

# Initialization code to run before you start your work

Click on the cell below and then type Shift-Return to execute it.

In [1]:
import numpy as np
import math

from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
 
# Draws a polygon in the xy-plane if you give it a list [[x1,y1], [x2,y2], ... ] of coordinates of the verties   
def Plot2dPolygon(list_of_vertices):   
    p = figure(width=400, height=400, title="Polygon")
    xcoords = [list_of_vertices[i][0] for i in range(len(list_of_vertices))] + [list_of_vertices[0][0]]
    ycoords = [list_of_vertices[i][1] for i in range(len(list_of_vertices))] + [list_of_vertices[0][0]]
    p.line(xcoords, ycoords, line_width=2)
    show(p)
    
# Input is a list of 2d vectors to be plotted.
def Plot2dVectors(list_of_endpoints):   
    p = figure(width=400, height=400, title="Vectors!")
    for vect in list_of_endpoints:
        xcoords = [0,vect[0]]
        ycoords = [0,vect[1]]
        p.line(xcoords, ycoords, line_width=2)
        p.circle([vect[0]], [vect[1]], color='red', size=2)
    show(p)

# Input is a list of 2d vectors as (x,y) coordinates of points to be plotted.
def Plot2dDots(list_of_points):   
    TOOLTIPS = [("(x,y)", "($x, $y)")]
    p = figure(width=600, height=600, title="Some points",tooltips=TOOLTIPS)
    D = np.array(list_of_points).T
    p.circle(D[0], D[1], color='blue', size=6)
    show(p)
    
        
# Input is a list of 3d vectors
def Plot3dVectors(list):
    plt.figsize(6,4)
    ax = plt.axes(projection = '3d')
    for vect in list:
        ax.plot([0,vect[0]], [0,vect[1]], [0,vect[2]],color='blue')
    ax.plot([0,0], [-10,10], [0,0], 'g--')
    ax.plot([-10,10], [0, 0], [0,0], 'g--')
    ax.plot([0,0], [0, 0], [-10,10], 'g--')
    plt.draw()
    plt.show()    

def Plot3dDots(list):       
    limit=20
    plt.figure(figsize=(10, 8), dpi=80)
    ax = plt.axes(projection = '3d')
    ax.set_xlim(-limit,limit)
    ax.set_ylim(-limit,limit)
    ax.set_zlim(-limit,limit)

    # Draw the shadow
    ax.scatter3D([item[0] for item in list], [item[1] for item in list], [-20 for item in list], color='grey')

    # Draw the coordinate axes
    ax.plot([0,0], [-limit,limit], [0,0], 'g--')
    ax.plot([-limit,limit], [0, 0], [0,0], 'g--')
    ax.plot([0,0], [0, 0], [-limit,limit], 'g--')

    # Now draw the points
    ax.scatter3D([item[0] for item in list], [item[1] for item in list], [item[2] for item in list], c=[item[2] for item in list])

    plt.draw()
    plt.show()    




Loading BokehJS ...

---
# Exercise 1

Suppose we have a discrete dynamical system $\vec{v}_{t+1} = A \vec{v}_t$ where 
$$A = \begin{pmatrix} -0.475 & -0.7875\\ -1.05   &  0.575 \end{pmatrix}.$$

1. Suppose we start with initial state $\vec{v}_0 = \begin{pmatrix} 1 \\ 0 \end{pmatrix}$.  Calculate $\vec{v}_1, \vec{v}_2, \vec{v}_3$.
2. Now suppose we start with initial state $\vec{v}_0 = \begin{pmatrix} -1 \\ 2 \end{pmatrix}$.  Calculate $\vec{v}_1, \vec{v}_2, \vec{v}_3$.
3. Write a Python function that takes an initial state $\vec{v}_0$ as input and returns a list of the first 50 vectors $\vec{v}_i$.
4. Use the function **Plot2dDots(..)** (defined above in the initialisation block) to plot your points.  Try a few different initial vectors.
5. Make a plot comparing the function $f(t)=e^{(1.1)t}$ with the $g(t)=|v_t|$ for some initial vectors.  You can use Plot2dDots(..) if you give it a list of pairs [[1,f(1)], [2, f(2)], [3, f(3)], ...]
 
6. Use **np.linalg.eig(A)** to find the eigenvectors and eigenvalues of A. This function will return a list with two elements.  The first element is a NumPy vector of eigenvalues $(\lambda_1, \lambda_2)$ (we have a $2\times2$ matrix, so in this case there are just 2 of them), and the second element is a matrix whose columns are the eigenvectors of $A$. 
7. Find an initial state $\vec{v}_0 \in \mathbb{R}^2$ that such that the length $|\vec{v}_t|$ stays the same as $t \to \infty$.  You might need to refer back to Workshop 6 to recall how to extract a column from a matrix.  
8. Can you find a $\vec{v}_0$ such that $|\vec{v}_t| = 10$ for all $t$?
9. Conceptual challenge: If you pick a random initial state, is it likely to stay the same length, shrink to zero, or grow to infinite length as $t\to \infty$?
10. Make a plot comparing the function $f(t)=e^{(1.1)t}$ with the $g(t)=|v_t|$ for 



In [ ]:
# You can write code here if you want.



---
# Exercise 2

Most insects have a life-cycle where they start as eggs, then they hatch out as larvae, then they pupate, and finally they emerge as an adult.
Consider a very simplistic model of some kind of insect:
* $E_t$ = number of eggs
* $L_t$ = number of larvae
* $P_t$ =  number of pupae
* $A_t$ = number of adults

We measure time $t$ in months.  Each month, the following happens:
* Half the eggs hatch into larvae.
* Half the larvae make a cocoon and pupate.
* 1/4 of the pupae finish and emerge as adults.
* 1/2 of the adults die, and 1/2 of them manage to lay an egg.

1. If we represent the state of this insect population by a vector $\vec{v}_t = \begin{pmatrix}E_t \\ L_t\\ P_t\\ A_t \end{pmatrix}$, write a matrix $M$ such that $M \vec{v}_t = \vec{v}_{t+1}$.
2. If we start with just 10 eggs, calculate how the population develops over the course of a year.
3. Find a number $\alpha$ such that $A_t$ looks approximately like the exponential function $e^{\alpha t}$.

In [ ]:
# Write some Python code here


---

# Congratulations!

You have come to the end of the material for this module.  Now you have some time to go back over the previous workshops and work on any exercises that you haven't finished yet.